In [3]:
#fed_learning_all_losses = pd.read_csv("losses/fed_learning_stats_64.csv")

In [1]:
import os
import time
import torch

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from torch.utils.data import DataLoader

from models import ShallowNN
from utils import load_file
from evals import evaluate,evaluate_mae_with_confidence,influence
from evals import euclidean_distance, manhattan_distance, pairwise_euclidean_distance, accumulated_proximity, full_accumulated_proximity

features = 197
batch_size = 64
loss_fn = torch.nn.L1Loss()

In [4]:
"""
fig,axes = plt.subplots(nrows=4,ncols=6,figsize=(60, 30))
for client in client_ids:
    plot_id = client.split("_")
    fed_losses = fed_learning_all_losses.loc[fed_learning_all_losses['client_id'] == client].reset_index().drop(columns=['index','training_round','client_id','fed_train'])
    fed_losses = fed_losses.rename(columns={"validation_loss": "fed_val"})
    iso_losses = pd.read_csv("losses/batch64_client_"+str(client)+".csv").drop(columns=["iso_train"])
    df = pd.concat([fed_losses, iso_losses], axis=1)
    df.plot(ax=axes[int(plot_id[0])][int(plot_id[1])],title=client)
"""

'\nfig,axes = plt.subplots(nrows=4,ncols=6,figsize=(60, 30))\nfor client in client_ids:\n    plot_id = client.split("_")\n    fed_losses = fed_learning_all_losses.loc[fed_learning_all_losses[\'client_id\'] == client].reset_index().drop(columns=[\'index\',\'training_round\',\'client_id\',\'fed_train\'])\n    fed_losses = fed_losses.rename(columns={"validation_loss": "fed_val"})\n    iso_losses = pd.read_csv("losses/batch64_client_"+str(client)+".csv").drop(columns=["iso_train"])\n    df = pd.concat([fed_losses, iso_losses], axis=1)\n    df.plot(ax=axes[int(plot_id[0])][int(plot_id[1])],title=client)\n'

In [2]:
eval_list = []
for client in client_ids:
    
    val_data_path =  "testpt/"+str(client)+".pt"
    val_set = torch.load(val_data_path)
    val_loader = DataLoader(val_set, batch_size, shuffle = True)
    
    isolated_model_path = "checkpt/isolated/batch64_client_"+str(client)+".pth"
    isolated_model =  ShallowNN(features)
    isolated_model.load_state_dict(torch.load(isolated_model_path))
    
    _, _ , isolated_mae = evaluate(isolated_model, val_loader, loss_fn)
    _, _ , global_mae = evaluate(global_model,val_loader,loss_fn)
    
    eval_dict = {"client_id":client, "Isolated Model MAE": round(isolated_mae, 4), "Federated Model MAE" :round(global_mae, 4)}
    eval_list.append(eval_dict)
    
eval_df = pd.DataFrame.from_dict(eval_list)
eval_df["clients"] = [i for i in range(1,25)] 

NameError: name 'client_ids' is not defined

In [ ]:
ax = eval_df.plot.bar(x="clients",figsize=(20, 10))
ax.set_ylabel("Mean Absolute Error for Validation",fontdict={'fontsize':13})
ax.set_xlabel("Client IDs",fontdict={'fontsize':13})
ax.legend(fontsize=15, loc="upper right")

In [ ]:
ben_list = []
for index, row in eval_df.iterrows():
    ben_list.append(round(row['Isolated Model MAE']-row['Federated Model MAE'],4))
eval_df["Benefict"] = ben_list

In [ ]:
eval_df.to_csv("insights/benifict.csv", index= False)

In [ ]:
eval_df.plot.bar(x="clients",y="Benefict", figsize=(8, 4))

In [ ]:
def influence(global_model, influenced_model):
    eval_list = []
    for client in client_ids:

        val_data_path =  "testpt/"+str(client)+".pt"
        val_set = torch.load(val_data_path)
        val_loader = DataLoader(val_set, batch_size, shuffle = True)

        global_loss, _ , global_mae = evaluate(global_model,val_loader,loss_fn)
        influenced_loss, _ , influenced_mae = evaluate(influenced_model, val_loader, loss_fn)

        eval_dict = {"client_id":client, "global_mae": round(global_mae, 4), 
                     "influenced_mae" :round(influenced_mae, 4),
                    "global_loss": round(global_loss,4),
                    "influenced_loss": round(influenced_loss,4)}
        
        eval_list.append(eval_dict)

    eval_df = pd.DataFrame.from_dict(eval_list)
    
    influence = round(eval_df["global_mae"].sum() - eval_df["influenced_mae"].sum() , 4)
    loss_influence = round(eval_df["global_loss"].sum() - eval_df["influenced_loss"].sum() , 4)
    return eval_df, influence,loss_influence

In [ ]:
inf_val = []
loss_inf_val = []
for client in client_ids:
    model = ShallowNN(features)
    model.load_state_dict(torch.load('checkpt/influence/' + str(client)+ '_fedl_global_500.pth'))
    df, inf,loss_inf = influence(global_model,model)
    inf_val.append(abs(inf))
    loss_inf_val.append(abs(loss_inf))
    #df.to_csv("influence2_"+str(client) + ".csv", index=False)

In [ ]:
data = {"client id": client_ids, "inf_val": inf_val, "loss_inf_val":loss_inf_val}
data = pd.DataFrame(data)
data.to_csv("insights/influence_with_batch_mae.csv" , index=False)